In [6]:
!python.exe -m pip install --upgrade pip
!pip install SPARQLWrapper
!pip install rdflib
!pip install mysql
!pip install ttkbootstrap
!pip install mysql-connector-python

In [4]:
from SPARQLWrapper import SPARQLWrapper, JSON
import urllib.request
import ttkbootstrap as ttk
from ttkbootstrap.constants import *
from ttkbootstrap.scrolled import ScrolledText
import logging
import mysql.connector
from tkinter import messagebox

# Configure logging
logging.basicConfig(level=logging.INFO)

# MySQL database configuration
DB_CONFIG = {
    "host": "localhost",
    "user": "root",  # Replace with your MySQL username
    "password": "",  # Replace with your MySQL password
    "database": "sparql_queries"
}

def connect_to_db():
    """Connect to the MySQL database."""
    try:
        connection = mysql.connector.connect(**DB_CONFIG)
        logging.info("Connected to MySQL database.")
        return connection
    except mysql.connector.Error as e:
        logging.error(f"Error connecting to MySQL: {e}")
        messagebox.showerror("Database Error", f"Could not connect to the database: {e}")
        return None

def test_https():
    """Test HTTPS connectivity to the SPARQL endpoint."""
    try:
        urllib.request.urlopen("https://dbpedia.org/sparql")
        logging.info("HTTPS connectivity test passed.")
        return True
    except Exception as e:
        logging.error(f"HTTPS failed: {e}")
        messagebox.showerror("HTTPS Error", f"HTTPS failed: {e}")
        return False

def query_dbpedia(endpoint_url, query):
    """Execute a SPARQL query against the given endpoint."""
    endpoint = SPARQLWrapper(endpoint_url)
    endpoint.setQuery(query)
    endpoint.setReturnFormat(JSON)
    try:
        results = endpoint.query().convert()
        logging.info("SPARQL query executed successfully.")
        return results
    except Exception as e:
        logging.error(f"SPARQL query failed: {e}")
        messagebox.showerror("SPARQL Error", f"An error occurred: {e}")
        return None

def process_results(results):
    """Process and display SPARQL query results."""
    if not results:
        return "No results to process."
    
    result_text = "Results:\n"
    for result in results["results"]["bindings"]:
        for key, value in result.items():
            result_text += f"{key}: {value['value']}\n"
        result_text += "---\n"
    return result_text

def on_execute_query(query_name, query, result_display):
    """Handle executing a query and displaying results."""
    if not test_https():
        return
    results = query_dbpedia("https://dbpedia.org/sparql", query)
    result_text = process_results(results)
    result_display.delete("1.0", "end")  # Clear previous results
    result_display.insert("end", result_text)  # Display new results

def load_queries_from_db(query_menu):
    """Load queries from the MySQL database and populate the dropdown."""
    connection = connect_to_db()
    if not connection:
        return

    try:
        cursor = connection.cursor()
        cursor.execute("SELECT query_name FROM queries")
        query_names = [row[0] for row in cursor.fetchall()]
        query_menu['values'] = query_names  # Populate the Combobox with query names
        if query_names:
            query_menu.current(0)  # Set the first query as the default
        logging.info("Queries loaded from database.")
    except mysql.connector.Error as e:
        logging.error(f"Error loading queries from database: {e}")
        messagebox.showerror("Database Error", f"Could not load queries: {e}")
    finally:
        cursor.close()
        connection.close()

def get_query_text_from_db(query_name):
    """Get the query text from the database based on the query name."""
    connection = connect_to_db()
    if not connection:
        return None

    try:
        cursor = connection.cursor()
        cursor.execute("SELECT query_text FROM queries WHERE query_name = %s", (query_name,))
        result = cursor.fetchone()
        if result:
            return result[0]
        else:
            messagebox.showerror("Error", f"Query '{query_name}' not found in the database.")
            return None
    except mysql.connector.Error as e:
        logging.error(f"Error retrieving query from database: {e}")
        messagebox.showerror("Database Error", f"Could not retrieve query: {e}")
        return None
    finally:
        cursor.close()
        connection.close()

def on_add_query(query_name_entry, query_text_entry, query_menu):
    """Add a new query to the database and update the dropdown."""
    query_name = query_name_entry.get().strip()
    query_text = query_text_entry.get("1.0", "end").strip()

    if query_name and query_text:
        connection = connect_to_db()
        if not connection:
            return

        try:
            cursor = connection.cursor()
            cursor.execute("INSERT INTO queries (query_name, query_text) VALUES (%s, %s)", (query_name, query_text))
            connection.commit()
            logging.info(f"Query '{query_name}' added to the database.")
            messagebox.showinfo("Success", f"Query '{query_name}' added successfully!")
            load_queries_from_db(query_menu)  # Refresh the dropdown
        except mysql.connector.Error as e:
            logging.error(f"Error adding query to database: {e}")
            messagebox.showerror("Database Error", f"Could not add query: {e}")
        finally:
            cursor.close()
            connection.close()
    else:
        messagebox.showerror("Error", "Please provide both a query name and the query text.")

def on_delete_query(selected_query, query_menu):
    """Delete the selected query from the database and update the dropdown."""
    query_name = selected_query.get()

    if not query_name:
        messagebox.showerror("Error", "No query selected.")
        return

    connection = connect_to_db()
    if not connection:
        return

    try:
        cursor = connection.cursor()
        cursor.execute("DELETE FROM queries WHERE query_name = %s", (query_name,))
        connection.commit()
        logging.info(f"Query '{query_name}' deleted from the database.")
        messagebox.showinfo("Success", f"Query '{query_name}' deleted successfully!")
        load_queries_from_db(query_menu)  # Refresh the dropdown
    except mysql.connector.Error as e:
        logging.error(f"Error deleting query from database: {e}")
        messagebox.showerror("Database Error", f"Could not delete query: {e}")
    finally:
        cursor.close()
        connection.close()

def create_gui():
    """Create the styled GUI for the application."""
    # Root window with ttkbootstrap theme
    app = ttk.Window(themename="solar")
    app.title("SPARQL Query Executor")
    app.geometry("800x600")

    # Frame for query selection
    query_frame = ttk.Frame(app, padding=10)
    query_frame.pack(fill=X, pady=5)

    ttk.Label(query_frame, text="Select Query:", font=("Helvetica", 12)).pack(side=LEFT, padx=5)
    selected_query = ttk.StringVar()
    query_menu = ttk.Combobox(query_frame, textvariable=selected_query, width=50, bootstyle="info")
    query_menu.pack(side=LEFT, fill=X, padx=5, expand=True)

    execute_button = ttk.Button(query_frame, text="Execute Query", command=lambda: on_execute_query(
        selected_query.get(),
        get_query_text_from_db(selected_query.get()),
        result_display
    ), bootstyle=SUCCESS)
    execute_button.pack(side=LEFT, padx=5)

    delete_button = ttk.Button(query_frame, text="Delete Query", command=lambda: on_delete_query(selected_query, query_menu), bootstyle=DANGER)
    delete_button.pack(side=LEFT, padx=5)

    # Result display area
    result_display = ScrolledText(app, width=80, height=20, bootstyle="secondary")
    result_display.pack(fill=BOTH, expand=True, pady=10, padx=10)

    # Adding Query Frame
    add_query_frame = ttk.Labelframe(app, text="Add New Query", padding=10)
    add_query_frame.pack(fill=X, pady=10, padx=10)

    ttk.Label(add_query_frame, text="Query Name:", font=("Helvetica", 12)).grid(row=0, column=0, sticky=W, pady=5)
    query_name_entry = ttk.Entry(add_query_frame, width=40)
    query_name_entry.grid(row=0, column=1, pady=5, padx=5)

    ttk.Label(add_query_frame, text="Query Text:", font=("Helvetica", 12)).grid(row=1, column=0, sticky=W, pady=5)
    query_text_entry = ttk.Text(add_query_frame, height=5, width=60)
    query_text_entry.grid(row=1, column=1, pady=5, padx=5)

    add_query_button = ttk.Button(add_query_frame, text="Add Query", command=lambda: on_add_query(query_name_entry, query_text_entry, query_menu), bootstyle="info")
    add_query_button.grid(row=2, column=0, columnspan=2, pady=10)

    # Load queries from the database
    load_queries_from_db(query_menu)

    app.mainloop()

if __name__ == "__main__":
    create_gui()

INFO:root:Connected to MySQL database.
INFO:root:Queries loaded from database.
INFO:root:Connected to MySQL database.
INFO:root:HTTPS connectivity test passed.
INFO:root:SPARQL query executed successfully.
INFO:root:Connected to MySQL database.
INFO:root:HTTPS connectivity test passed.
INFO:root:SPARQL query executed successfully.
INFO:root:Connected to MySQL database.
INFO:root:Query 'List of famous actors' deleted from the database.
INFO:root:Connected to MySQL database.
INFO:root:Queries loaded from database.
INFO:root:Connected to MySQL database.
INFO:root:HTTPS connectivity test passed.
INFO:root:SPARQL query executed successfully.
INFO:root:Connected to MySQL database.
INFO:root:HTTPS connectivity test passed.
INFO:root:SPARQL query executed successfully.


KeyboardInterrupt: 